In [ ]:
%pip install evaluate
%pip install rouge_score
%pip install sacrebleu
%pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=01d69c4723a3788613e00e1cbe806b328134cb43be71a73d34003755081b5c3e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# из библиотеки google.colab импортируем класс files
from google.colab import files

# создаем объект этого класса, применяем метод .upload()
uploaded = files.upload()

In [ ]:
!cp /content/drive/MyDrive/hackathon_hse25-main/prepocess_calculate/func_to_call.py .
!cp /content/drive/MyDrive/hackathon_hse25-main/prepocess_calculate/metrics.py .

In [ ]:
import json
from func_to_call import parse_all_data, parse_data_with_time


data_v1 = parse_data_with_time('/content/drive/MyDrive/hackathon_hse25-main/prepocess_calculate/datasets/train_set.json')

data_v2 = parse_data_with_time('/content/drive/MyDrive/hackathon_hse25-main/prepocess_calculate/datasets/val_set.json')

data_v1.extend(data_v2)
#print(data_v1.append(data_v2))

with open('parsed_tuning.json', 'w', encoding='utf-8') as f:
    json.dump(data_v1, f, ensure_ascii=False)

import json

with open('parsed_tuning.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)

formatted_data = []

from metrics import ValidatorSimple


vs = ValidatorSimple(neural=True)


for item in training_data:
    contexts = [ctx['text'] for ctx in item['contexts']]

    if item['winner'] == 'Saiga':
        score = vs.score_sample(item['giga_answer'], item['saiga_answer'], contexts)
        formatted_data.append({
            'selected_role': item['selected_role'],
            'campus': item['campus'],
            'education_level': item['education_level'],
            'question_category': item['question_category'],
            'user_filters': item['user_filters'],
            'question_filters': item['question_filters'],
            "question": item['user_question'],
            "answer": item['giga_answer'],
            "ground_truth": item['saiga_answer'],
            "contexts": contexts,
            "source": "saiga",
            "rating": "good" if item['winner'] in ['Saiga', 'Оба хорошо'] else "bad",
            "response_time": item["response_time"],
            "context_recall": score["context_recall"][0],
            "context_precision": score["context_precision"][0],
            "answer_correctness_literal": score["answer_correctness_literal"][0],
            "answer_correctness_neural": score["answer_correctness_neural"][0][0]
        })

    elif item['winner'] == 'GigaChat':
        score = vs.score_sample(item['saiga_answer'], item['giga_answer'], contexts)
        formatted_data.append({
            'selected_role': item['selected_role'],
            'campus': item['campus'],
            'education_level': item['education_level'],
            'question_category': item['question_category'],
            'user_filters': item['user_filters'],
            'question_filters': item['question_filters'],
            "question": item['user_question'],
            "answer": item['saiga_answer'],
            "ground_truth": item['giga_answer'],
            "contexts": contexts,
            "source": "giga",
            "rating": "good" if item['winner'] in ['GigaChat', 'Оба хорошо'] else "bad",
            "response_time": item["response_time"],
            "context_recall": score["context_recall"][0],
            "context_precision": score["context_precision"][0],
            "answer_correctness_literal": score["answer_correctness_literal"][0],
            "answer_correctness_neural": score["answer_correctness_neural"][0][0]
        })

    elif item['winner'] == 'Оба хорошо':
        score_1 = vs.score_sample(item['saiga_answer'], item['giga_answer'], contexts)
        score_2 = vs.score_sample(item['giga_answer'], item['saiga_answer'], contexts)
        formatted_data.extend([
            {
                'selected_role': item['selected_role'],
                'campus': item['campus'],
                'education_level': item['education_level'],
                'question_category': item['question_category'],
                'user_filters': item['user_filters'],
                'question_filters': item['question_filters'],
                "question": item['user_question'],
                "answer": item['saiga_answer'],
                "ground_truth": item['giga_answer'],
                "contexts": contexts,
                "source": "saiga",
                "rating": "good",
                "response_time": item["response_time"],
                "context_recall": score_1["context_recall"][0],
                "context_precision": score_1["context_precision"][0],
                "answer_correctness_literal": score_1["answer_correctness_literal"][0],
                "answer_correctness_neural": score_1["answer_correctness_neural"][0][0]
            },
            {
                'selected_role': item['selected_role'],
                'campus': item['campus'],
                'education_level': item['education_level'],
                'question_category': item['question_category'],
                'user_filters': item['user_filters'],
                'question_filters': item['question_filters'],
                "question": item['user_question'],
                "answer": item['giga_answer'],
                "ground_truth": item['saiga_answer'],
                "contexts": contexts,
                "source": "giga",
                "rating": "good",
                "response_time": item["response_time"],
                "context_recall": score_2["context_recall"][0],
                "context_precision": score_2["context_precision"][0],
                "answer_correctness_literal": score_2["answer_correctness_literal"][0],
                "answer_correctness_neural": score_2["answer_correctness_neural"][0][0]
            }
        ])

    elif item['winner'] == 'Оба плохо':
        score_1 = vs.score_sample(item['saiga_answer'], item['giga_answer'], contexts)
        score_2 = vs.score_sample(item['giga_answer'], item['saiga_answer'], contexts)
        formatted_data.extend([
            {
                'selected_role': item['selected_role'],
                'campus': item['campus'],
                'education_level': item['education_level'],
                'question_category': item['question_category'],
                'user_filters': item['user_filters'],
                'question_filters': item['question_filters'],
                "question": item['user_question'],
                "answer": item['saiga_answer'],
                "ground_truth": item['giga_answer'],
                "contexts": contexts,
                "source": "saiga",
                "rating": "bad",
                "response_time": item["response_time"],
                "context_recall": score_1["context_recall"][0],
                "context_precision": score_1["context_precision"][0],
                "answer_correctness_literal": score_1["answer_correctness_literal"][0],
                "answer_correctness_neural": score_1["answer_correctness_neural"][0][0]
            },
            {
                'selected_role': item['selected_role'],
                'campus': item['campus'],
                'education_level': item['education_level'],
                'question_category': item['question_category'],
                'user_filters': item['user_filters'],
                'question_filters': item['question_filters'],
                "question": item['user_question'],
                "answer": item['giga_answer'],
                "ground_truth": item['saiga_answer'],
                "contexts": contexts,
                "source": "giga",
                "rating": "bad",
                "response_time": item["response_time"],
                "context_recall": score_2["context_recall"][0],
                "context_precision": score_2["context_precision"][0],
                "answer_correctness_literal": score_2["answer_correctness_literal"][0],
                "answer_correctness_neural": score_2["answer_correctness_neural"][0][0]
            }
        ])

    else:
        score = vs.score_sample(item['giga_answer'], item['saiga_answer'], contexts)
        formatted_data.append({
            'selected_role': item['selected_role'],
            'campus': item['campus'],
            'education_level': item['education_level'],
            'question_category': item['question_category'],
            'user_filters': item['user_filters'],
            'question_filters': item['question_filters'],
            "question": item['user_question'],
            "answer": item['saiga_answer'],
            "ground_truth": item['giga_answer'],
            "contexts": contexts,
            "source": "unknown",
            "rating": "neutral",
            "response_time": item["response_time"],
            "context_recall": score["context_recall"][0],
            "context_precision": score["context_precision"][0],
            "answer_correctness_literal": score["answer_correctness_literal"][0],
            "answer_correctness_neural": score["answer_correctness_neural"][0][0]
        })


import pandas as pd

# в сумме ~ 800
dataset = pd.DataFrame(formatted_data)
"""dataset = pd.DataFrame({
    "question": [
        "Какие документы регулируют порядок обслуживания студентов в столовой?",
        "Каковы основные этапы прохождения учебной практики?",
    ],

    "answer": [
        "Я ГЛУПИ МОДЕЛЬ НЕ УМЕЮ...",
        "А Я УМЕЮ...", #ответы вашей модели
    ],
    "ground_truth": [
        "Порядок организации обслуживания регулируется следующими документами...",
        "Основные этапы включают: 1. Размещение программы...", #эталонные ответы
    ],
    "contexts": [
        [
            "сотрудника НИУ ВШЭ с указанием руководителя...",
            "Центр сервиса «Студент» – Национальный исследовательский университет...",
            # остальные контексты для первого вопроса
        ],
        [
            "уточной аттестации и текущего контроля успеваемости студентов...",
            "Траектории обучения в бакалавриате...",
            # ... контексты для второго вопроса
        ],
    ]
})"""

"""from metrics import ValidatorSimple


vs = ValidatorSimple(neural=True)
vs.validate_rag(dataset)"""

print(dataset)

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


     selected_role           campus education_level  \
0          Студент  Нижний Новгород     Бакалавриат   
1          Студент           Москва     Бакалавриат   
2          Студент  Нижний Новгород     Бакалавриат   
3          Студент           Москва     Бакалавриат   
4          Студент  Нижний Новгород     Бакалавриат   
...            ...              ...             ...   
1283       Студент  Нижний Новгород     Бакалавриат   
1284       Студент  Нижний Новгород     Бакалавриат   
1285       Студент           Москва     Бакалавриат   
1286       Студент           Москва     Бакалавриат   
1287       Студент           Москва     Бакалавриат   

                    question_category                    user_filters  \
0                               Наука  [Нижний Новгород, бакалавриат]   
1                  Социальные вопросы           [Москва, бакалавриат]   
2                              Военка  [Нижний Новгород, бакалавриат]   
3                  Финансовые вопросы          

In [ ]:
dataset_new = pd.DataFrame(formatted_data).to_dict('list')

In [ ]:
with open('end_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(dataset_new, f, ensure_ascii=False)

In [ ]:
import numpy as np
dataset2 = pd.DataFrame(dataset_new)
print(dataset2.shape)
print(np.mean(dataset2["context_recall"]))
print(np.mean(dataset2["context_precision"]))
print(np.mean(dataset2["answer_correctness_literal"]))
print(np.mean(dataset2["answer_correctness_neural"]))
dataset2.head()

(1288, 17)
0.10008550320950892
0.07585244567673413
32.66223383653942
0.7142675065013192


,selected_role,campus,education_level,question_category,user_filters,question_filters,question,answer,ground_truth,contexts,source,rating,response_time,context_recall,context_precision,answer_correctness_literal,answer_correctness_neural
0,Студент,Нижний Новгород,Бакалавриат,Наука,"[Нижний Новгород, бакалавриат]",[Наука],"Для чего нужно подробное изучение науки, прове...","Я понял ваш вопрос. Вы хотите узнать, почему н...","Подробное изучение науки, проведение исследова...",[Конкурсы игранты НИУВШЭ Конкурсы и гранты НИУ...,giga,good,3.517504,0.083333,0.050694,36.110141,0.759762
1,Студент,Москва,Бакалавриат,Социальные вопросы,"[Москва, бакалавриат]","[Социальные вопросы, Безопасность, Общежития]",Нужно ли подавать какие-то дополнительные доку...,Дополнительные документы для продления социаль...,Для продления социальной стипендии необходимо ...,[Материальная помощь и материальная поддержка ...,saiga,good,3.763305,0.100000,0.062192,32.441799,0.718018
2,Студент,Нижний Новгород,Бакалавриат,Военка,"[Нижний Новгород, бакалавриат]",[ВУЦ],"я 120см, смогу ли я попасть на военку?",Информация о требованиях для участия в военной...,Я понял ваш страх по поводу возможности поступ...,[енной службы с зачислением в запас Вооруженны...,saiga,good,1.813993,0.103202,0.071666,34.258986,0.714013
3,Студент,Москва,Бакалавриат,Финансовые вопросы,"[Москва, бакалавриат]",[Деньги],Нужно ли подавать какие-то дополнительные доку...,Дополнительные документы для продления социаль...,Для продления социальной стипендии необходимо ...,[Материальная помощь и материальная поддержка ...,saiga,good,4.789084,0.100000,0.062192,32.441799,0.718018
4,Студент,Нижний Новгород,Бакалавриат,Цифровые сервисы и Техподдержка,"[Нижний Новгород, бакалавриат]",[Цифровые системы],Если во время прохождения теста в смарт лмс пр...,"Я понял ваш вопрос о том, что случилось с тест...",Если во время прохождения теста в смарт ЛМС пр...,[Learning management system (LMS) & SmartLMS L...,saiga,good,3.572672,0.042254,0.037786,20.540944,0.681225
